### **Adversarial Attacks on LISA Dataset**

Parameters for the **inconspicuous** attack on LISA data:
* mask threshold 0.1
* mask penalty for using colors (beta1) 0.1
* mask regularization for noise (beta2) 0.01
* mask deviation penalty (beta3) 4.0
* attack LR 0.01
* attack penalty for using colors (beta1) 1.0
* attack deviation penalty (beta3) 4.0
* 20000 attack training epochs


Parameters for the **visible** attack on LISA data:
* mask threshold 0.01
* mask penalty for using colors (beta1) 0.0
* mask regularization for noise (beta2) 0.0
* mask deviation penalty (beta3) 1.0
* attack LR 0.0001
* attack penalty for using colors (beta1) 0.0
* attack deviation penalty (beta3) 1.0
* 50000 attack training epochs


For both attacks:
* 1000 mask training epochs
* mask initialization LR 0.01

* attack batch size 8

* augmentation is not used for initialization, but used for attack

For the attack on class 12 (stop), the target class is 11 (speed limit 45). For the attack on class 9 (speed limit 30), the target class is 12 (stop).


## **Invonspicuous Attack**

In [33]:
import pickle
import os
import json
from models import *
from utils import *
from torchvision.models import mobilenet_v2, efficientnet_b0, densenet121, shufflenet_v2_x1_0

# Parameters of LISA dataset (you can leave this as it is)
with open('classes.json', 'r') as config:
    params = json.load(config)
    params['CLASS_N'] = params['LISA']['class_n']
    params['DEVICE'] = params['device']
    params['LABELS'] = params['LISA']['labels']

    
model_types = ['CNNsmall', 'CNNlarge', 'Transformer', 'ResNet18', 'MobileNetv2', 'DenseNet121', 'ShuffleNetv2x1', "EfficientNet"]


model_type = "EfficientNet"
print(model_type)

params['MODEL_TYPE'] = model_type
# Directory where output files will be stored
params['OUTPUT_DIR'] = f"attacks/inconspicuous{params['MODEL_TYPE']}LISA_ATTACK"

# --- PATHS INPUT---
# Path to weights of baseline model to attack
params['PATH_MODEL'] = f"models/{model_type}LISA.pth"
# Set your hyperparameters (we add them to the params dictionary)

# --- GENERAL PARAMETERS ---
# Target class that should be recognized instead of orginal class of sign
params['TARGET_CLASS'] = 2

# Path of original sign to pertubate (should be in original printable size)
params['PATH_SIGN'] = 'attack_utils/lisa_30.jpg'
# Path to the sign mask (defines area of sign)
params['PATH_SIGN_MASK'] = 'attack_utils/lisa_30_mask.jpg'

# --- PATHS OUTPUT --- (ONLY CHANGE FILENAMES)
# Path where to store the pertubated sign (printing size)
params['PATH_PERT_SIGN_SMALL'] = os.path.join(params['OUTPUT_DIR'], 'pert_sign_small.png')
# Path where to store the pertubated sign (printing size)
params['PATH_PERT_SIGN_LARGE'] = os.path.join(params['OUTPUT_DIR'], 'pert_sign_large.png')
# Path where to store the sign in the size of the model input
params['PATH_ORIG_SIGN_SMALL'] = os.path.join(params['OUTPUT_DIR'], 'orig_sign_small_.png')
# Path where to store the last augmented sign
params['PATH_AUGMENTED_SIGN_SMALL'] = os.path.join(params['OUTPUT_DIR'], 'augmented_sign_small.png')
# Path where to store the noise in small size
params['PATH_NOISE_SMALL'] = os.path.join(params['OUTPUT_DIR'], 'noise_small.png')
# Path where to store the noise in printable size
params['PATH_NOISE_LARGE'] = os.path.join(params['OUTPUT_DIR'], 'noise_large.png')
# Path where to store the noise mask
params['PATH_NOISE_MASK_SMALL'] = os.path.join(params['OUTPUT_DIR'], 'noise_mask_small.png')
# Path where to store the noise mask
params['PATH_NOISE_MASK_LARGE'] = os.path.join(params['OUTPUT_DIR'], 'noise_mask_large.png')
# Paht where to store noise tensor
params['PATH_NOISE_TENSOR'] = os.path.join(params['OUTPUT_DIR'], 'noise_tensor.pt')

# --- PARAMETERS FOR INITIALIZATION ---
params['INIT_MASK_THRESHOLD'] = 0.1
# How much you want to penalty using colors
params['INIT_MASK_BETA_1'] = 0.1
# How much you want to regularize the ammount of noise
params['INIT_MASK_BETA_2'] = 0.01
# How much you want to penalty the deviation from to target class
params['INIT_MASK_BETA_3'] = 4
# Learning rate of adam for the mask initialization
params['INIT_MASK_LEARNING_RATE'] = 0.01
# How many epochs you want to train the initialization of the mask
params['INIT_MASK_EPOCHS'] = 1000
# Turn augmentation for initialization of mask on/off
params['INIT_MASK_AUGMENTATION'] = False


# PARAMETERS FOR ATTACK
# Learning rate of adam for the attack
params['ATTACK_LEARNING_RATE'] = 0.01
# Batch size for one optimization step
params['ATTACK_BATCH_SIZE'] = 8
# How much you want to penalty using colors in the attack
params['ATTACK_BETA_1'] = 1
# How much you want to regularize the ammount of noise in the attack
params['ATTACK_BETA_2'] = 0.0
# How much you want to penalty the deviation from to target class
params['ATTACK_BETA_3'] = 4
# How many epochs you want to train the attack
params['ATTACK_EPOCHS'] = 20000
# Turn augmentation on/off for attack
params['ATTACK_AUGMENTATION'] = True

EfficientNet


In [34]:
transform_to_tensor = transforms.Compose([transforms.Resize((32, 32)),
                                              transforms.ToTensor()])

orig_sign = torch.clamp(transform_to_tensor(Image.open(params['PATH_SIGN'])), 0, 1)

model = load_model(params)

print("Prediction on the original sign:")
test_on_tensor(model=model, tensor=orig_sign, params=params, labels=params['LABELS'])

Prediction on the original sign:
Class with the max confidence is  9: Speed Limit 30
Predict: 9 (9: Speed Limit 30) Confidence: 62.22227215766907%


In [ ]:
# Create output directory
if not os.path.exists(params['OUTPUT_DIR']):
  os.makedirs(params['OUTPUT_DIR'])

print(f"-----------------{model_type}----------------------")

# Train attack
train_attack(params)
# Test attack
test_attack(params)

## **Visible Attack**

In [ ]:
import os

# Parameters of LISA dataset (you can leave this as it is)
with open('classes.json', 'r') as config:
    params = json.load(config)
    params['CLASS_N'] = params['LISA']['class_n']
    params['DEVICE'] = params['device']
    params['LABELS'] = params['LISA']['labels']

model_types = ['CNNsmall', 'CNNlarge', 'Transformer', 'ResNet18', 'MobileNetv2', 'DenseNet121', 'ShuffleNetv2x1', "EfficientNet"]


model_type = "EfficientNet"
print(model_type)


# Set your hyperparameters (we add them to the params dictionary)

# --- GENERAL PARAMETERS ---
# Target class that should be recognized instead of orginal class of sign
params['TARGET_CLASS'] = 12
params['MODEL_TYPE'] = model_type
# Directory where output files will be stored
params['OUTPUT_DIR'] = f"attacks/visible{params['MODEL_TYPE']}LISA_ATTACK"

# --- PATHS INPUT---
# Path to weights of baseline model to attack
params['PATH_MODEL'] = f"models/{model_type}LISA.pth"
# Path of original sign to pertubate (should be in original printable size)
params['PATH_SIGN'] = 'attack_utils/lisa_30.jpg'
# Path to the sign mask (defines area of sign)
params['PATH_SIGN_MASK'] = 'attack_utils/lisa_30_mask.jpg'


# --- PATHS OUTPUT --- (ONLY CHANGE FILENAMES)
# Path where to store the pertubated sign (printing size)
params['PATH_PERT_SIGN_SMALL'] = os.path.join(params['OUTPUT_DIR'], 'pert_sign_small.png')
# Path where to store the pertubated sign (printing size)
params['PATH_PERT_SIGN_LARGE'] = os.path.join(params['OUTPUT_DIR'], 'pert_sign_large.png')
# Path where to store the sign in the size of the model input
params['PATH_ORIG_SIGN_SMALL'] = os.path.join(params['OUTPUT_DIR'], 'orig_sign_small_.png')
# Path where to store the last augmented sign
params['PATH_AUGMENTED_SIGN_SMALL'] = os.path.join(params['OUTPUT_DIR'], 'augmented_sign_small.png')
# Path where to store the noise in small size
params['PATH_NOISE_SMALL'] = os.path.join(params['OUTPUT_DIR'], 'noise_small.png')
# Path where to store the noise in printable size
params['PATH_NOISE_LARGE'] = os.path.join(params['OUTPUT_DIR'], 'noise_large.png')
# Path where to store the noise mask
params['PATH_NOISE_MASK_SMALL'] = os.path.join(params['OUTPUT_DIR'], 'noise_mask_small.png')
# Path where to store the noise mask
params['PATH_NOISE_MASK_LARGE'] = os.path.join(params['OUTPUT_DIR'], 'noise_mask_large.png')
# Paht where to store noise tensor
params['PATH_NOISE_TENSOR'] = os.path.join(params['OUTPUT_DIR'], 'noise_tensor.pt')

# --- PARAMETERS FOR INITIALIZATION ---
params['INIT_MASK_THRESHOLD'] = 0.01
# How much you want to penalty using colors
params['INIT_MASK_BETA_1'] = 0
# How much you want to regularize the ammount of noise
params['INIT_MASK_BETA_2'] = 0
# How much you want to penalty the deviation from to target class
params['INIT_MASK_BETA_3'] = 1
# Learning rate of adam for the mask initialization
params['INIT_MASK_LEARNING_RATE'] = 0.01
# How many epochs you want to train the initialization of the mask
params['INIT_MASK_EPOCHS'] = 1000
# Turn augmentation for initialization of mask on/off
params['INIT_MASK_AUGMENTATION'] = False


# PARAMETERS FOR ATTACK
# Learning rate of adam for the attack
params['ATTACK_LEARNING_RATE'] = 0.0001
# Batch size for one optimization step
params['ATTACK_BATCH_SIZE'] = 8
# How much you want to penalty using colors in the attack
params['ATTACK_BETA_1'] = 0.0
# How much you want to regularize the ammount of noise in the attack
params['ATTACK_BETA_2'] = 0.01
# How much you want to penalty the deviation from to target class
params['ATTACK_BETA_3'] = 1.0
# How many epochs you want to train the attack
params['ATTACK_EPOCHS'] = 20000
# Turn augmentation on/off for attack
params['ATTACK_AUGMENTATION'] = True

# Create output directory
if not os.path.exists(params['OUTPUT_DIR']):
  os.makedirs(params['OUTPUT_DIR'])

print(f"-----------------{model_type}----------------------")

# Train attack
train_attack(params)
# Test attack
test_attack(params)